In [3]:
# ================================================================================================================
#  Question Answering using Embeddings - GPT-3 to respond questions with insightful answers.

# https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipyn

# Comments:
# The level of answers look pretty good. I am wondering if there are a way to measure the accuracy 
# of these similarities.

# Including "context" improves the accuracy. I set the temperature = 0.

# We could convert our.pdf (.xml) files to JSON and loop through the volumes. As a context, we will use:
# Sections 1 - 1. EXPOSURE CHARACTERIZATION
# Section 2 -  2. CANCER IN HUMANS
# Section 6 - 6. EVALUATION AND RATIONALE

# What if we use the whole volume?

# The main disadvantage of using GPT is that we cannot access the parameters, and it is not free.

# Can we create a machine learning algorithm with comparable accuracy to GPT3 to execute this task?

# Step 1 (See example below): Create a dataset with Q&A using our free account on openAI API to supplement the current dataset.
# Step 2 - Build a ML model and use Step 1 to train a fine-tuning model.

#==================================================================================================================

import openai
import pandas as pd
import embeddings
import numpy as np
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003" #Expensive
EMBEDDING_MODEL = "text-embedding-ada-002" #Cheap

API_KEY = "sk-"  # supply your API key however you choose
openai.api_key = API_KEY


In [8]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
1.1 Identification of the agent
1.1.1 Introduction to the agent
Opium is a highly addictive narcotic drug
obtained from the juice (latex) of the unripe seedpod of the poppy plant (Papaver somniferum).
The latex requires minimal processing before
it can be consumed. The traditional practices
of latex processing vary from place to place and
may include air-drying, heat-drying, or boiling
(DEA, 1992, 2020a; Ray et al., 2006).
Chemically, opium is a complex mixture;
notably, it contains several alkaloids, including
morphine, codeine, noscapine, thebaine, and
papaverine. After extraction and purification
(Kalant, 1997), the alkaloids may also be used as
base material for the manufacture of semisynthetic opiate derivatives, such as heroin (from
morphine) and oxycodone (from thebaine)
(Kalant, 1997; Yaksh & Wallace, 2018). The
alkaloid components of opium or their derivatives are responsible for its analgesic, hypnotic,
antitussive, and antidiarrhoeal effects when
consumed (Labanca et al., 2018). There are also
many wholly synthetic opioid drugs available, for
example, fentanyl and methadone. These drugs
mimic the effects of opium on consumers but are
not manufactured from opium or its alkaloids
and may have chemically unrelated structures.
The agent considered in the present monograph is “opium consumption”. In this context,
the term opium includes raw, minimally refined,
and dross types, which are derived from the
poppy latex and comprise a complex chemical mixture. The agent definition also includes
contaminants that are integral to the complex
mixture (see Section 1.4.2(g)). The agent definition does not include the pure alkaloids that can
be extracted from opium (for example, morphine
or codeine), their semisynthetic modifications
such as heroin, or wholly synthetic opioid
compounds such as fentanyl.
1.1.2 Botanical and chemical data
(a) Nomenclature of opium and its source
plant
Botanical name: Papaver somniferum
Family: Papaveraceae
Subfamily: Papaveroideae
Common names: opium poppy, breadseed
poppy (Mahr, 2017; Labanca et al., 2018)
Opium (lachryma papaveris or “poppy
tears”, also called “raw opium”) is the dried latex
obtained from the unripe seedpods of the opium
poppy plants (INCB, 2019).
Although, as a complex mixture, minimally
processed opium has no chemical structure or
formula, a highly purified form of opium does
have a Chemical Abstracts Service (CAS) registry
number of 8008-60-4 (Drugs.com, 2019a).
1. EXPOSURE CHARACTERIZATION
IARC MONOGRAPHS – 126
44
European Community/List No.: 232-368-5
(ECHA, 2019), also known as “crude opium”
WHO Anatomical Therapeutic Chemical
codes: A07DA02, N02AA02 (WHOCC, 2019)
The nomenclature of certain opium products (e.g. as described in Section  1.4.1(c)) has
varied over time and by country (USDA, 2020).
For example, opium is called teriak in Persian,
afeen in Hindi, and afiyoon in Urdu, while
(minimally) refined (i.e. further processed, for
example, condensed) opium is known as chandu
in India and Myanmar (DEA, 1992). In the
Islamic Republic of Iran (hereafter referred to as
“Iran”), different types of opium are known by
different terms: teriak for raw opium, sukhteh or
sookhteh for opium dross, and shireh or shire for
refined opium (Khademi et al., 2012; Nikfarjam
et al., 2016). There is also a multitude of “street
names”, such as Aunti, Aunti Emma, Big O,
Black Pill, Chinese Molasses, Dopium, Dream
Gun, Fi-do-nie, Gee, Guma, Midnight Oil, and
Zero (DEA, 2020a).
(b) Description of the plant and its cultivation
P. somniferum (Fig. 1.1) is a dicotyledonous,
dialypetalous, superovaried annual herb,
30–150  cm long with a stem between 0.5 and
1.5 cm thick, blue-green leaves, and four white,
violet, or purple petals, which is cultivated in
temperate and subtropical regions. It is hardy and
may be grown without fertilizers or pesticides
(UNODC, 1953a; Ray et al., 2006; Pushpangadan
et al., 2012). Traditionally, most highland and
upland farmers in south-eastern Asia have not
used fertilizers for any of their crops, but opium
poppy farmers have started using both natural
and chemical fertilizers to increase opium poppy
yields in recent years. Chicken manure, human
faeces, or bat droppings are mixed into the
planting soil. Opium poppy seeds are planted by
the end of October. After 1 month, some plants
are removed and 8–18  plants/m2 are left. Most
opium poppy varieties in south-eastern Asia
produce three to five mature pods per plant.
Harvesting is done in February, about 2  weeks
after the flower petals fall from the pods (DEA,
1992). All parts of plants of the Papaver genus are
characterized by watery and milky latex, except
the seeds.
1.1.3 Opium composition and forms
(a) Opium composition
Opium has a complex chemical composition
consisting of sugars, proteins, fats, water, meconic
acid, plant wax, latex, gum, ammonia, sulfuric
and lactic acids, and at least 25 alkaloids (Fig. 1.2;
Pushpangadan et al., 2012; Labanca et al., 2018).
The composition of opium is further discussed
in Section 4.1. The alkaloids are categorized into
two main chemical classes: phenanthrenes and
benzylisoquinolines (Yaksh & Wallace, 2018).
The types and percentages of these alkaloids
differ widely in different poppy cultivars. Also,
differences in seed yield per plant and latex yield
per plant have been reported (Solanki et al., 2017;
Labanca et al., 2018). The principal phenanthrenes are morphine, codeine, and thebaine
(Yaksh & Wallace, 2018). Morphine is the most
abundant phenanthrene in opium. The metabolites of thebaine, an intermediate of morphine
biosynthesis in poppy plants, include thebaol
and oripavine (WHO, 2006; Megutnishvili et al.,
2018).
The principal benzylisoquinolines are papaverine and noscapine (Frick et al., 2005;
Beaudoin & Facchini, 2014; Labanca et al., 2018;
Yaksh & Wallace, 2018).
(b) Composition of different forms of opium
The main forms of opium are raw or crude
opium, opium dross, refined opium, and other
derivatives.
(i) Raw or crude opium
Raw or crude opium is the unprocessed
dried latex of the poppy seedpod and is a dark
and sticky substance (Fig.  1.3; Khademi et al.,
2012). Alkaloids are major components of crude
opium. Among all alkaloids, the most abundant
is morphine (10–12% of crude opium by weight).
The other alkaloids – including thebaine,
codeine, papaverine, noscapine, narcotine,
and narceine (UNODC, 1953a; Labanca et al.,
2018), together with morphine – make up about
25% of the weight of crude opium, as shown in
Table 1.1. Sterols such as cycloartenol and β-sitosterol (Malaveille et al., 1982) are also present
in raw opium. [The Working Group noted that
the expression of the morphine content of opium
as a percentage depends in part on the moisture
content.] After the opium has been collected, the
moisture content is usually ~30%. Commercial
opium usually has a moisture content of ~10–15%.
Opium that is apparently dry still retains considerable moisture (~6%) (UNODC, 1953a). Raw
opium is often adulterated with other substances,
for example, starch (37%) and strontium sulfate
were reported in Mexican opium (Sodi Pallares
& Meyran Garcia, 1954); arsenic in Indian
opium (Datta & Kaul, 1977; Narang et al., 1987);
and arsenic and strychnine in Sri Lankan opium
(Wijesekera et al., 1988). More recently, traded
opium in Iran has been shown to be contaminated with heavy metals such as lead, chromium,
cadmium, and thallium, and bacteria such as
Clostridium tetani, Enterobacter cloacae, and
Pantoea agglomerans (Ghaderi & Afshari, 2016;
Aghababaei et al., 2018; Ghane et al., 2018).
(ii) Opium dross
Dross is produced by scraping away the
tarry residues that accumulate on the inside
walls of the opium pipe as a result of incomplete 
combustion of raw opium. The dross is black,
dry, and granular (Meysamie et al., 2009). Each
gram of opium yields 0.3  g of dross (Siassi &
Fozouni, 1980). The summary content of the five
main alkaloids in opium dross is less than their
summary content in raw opium (see Table 1.1).
This pyrolysed opium (Meysamie et al., 2009),
unlike raw opium, contains primary aromatic
amines, heterocyclic (nitrogen-containing)
aromatic compounds, and polycyclic aromatic
hydrocarbons.
(iii) Refined opium or opium sap
Refined opium or opium sap is the (minimally) refined product of opium and is prepared
by boiling opium dross with or without raw
opium in water for several hours. The solution
is then filtered, insoluble parts are separated,
and the excess water is left to evaporate. The
final product is brown, sticky, and shiny (Kalant,
1997; Meysamie et al., 2009; Khademi et al., 2012;
Nikfarjam et al., 2016). The summary alkaloid
content in minimally refined opium is about the
same as in raw opium (see Table 1.1).
(iv) Other opium derivatives
Other opium derivatives include the pure
alkaloids like morphine and codeine; semisynthetic opiates like heroin, compact-heroin,
and buprenorphine; and crystal and synthetic
opioids like fentanyl, methadone, and pethidine.
This group of opium derivative drugs is outside
of the scope of this monograph (see Section 1.1.1,
Introduction to the agent).

Q: What is opium consumption?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Opium consumption is the consumption of raw, minimally refined, and dross types of opium, which are derived from the poppy latex and comprise a complex chemical mixture, including contaminants that are integral to the complex mixture. It does not include the pure alkaloids that can be extracted from opium (for example, morphine or codeine), their semisynthetic modifications such as heroin, or wholly synthetic opioid compounds such as fentanyl.'

In [6]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
6.1 Cancer in humans
There is sufficient evidence in humans for the
carcinogenicity of opium consumption. Opium
consumption causes cancers of the urinary
bladder, larynx, and lung. Positive associations
have been observed between opium consumption and cancers of the oesophagus, stomach,
pancreas, and pharynx.
6.2 Cancer in experimental animals
There is inadequate evidence in experimental
animals regarding the carcinogenicity of opium.
6.3 Mechanistic evidence
There is strong evidence in experimental
systems that opium, specifically sukhteh and
opium pyrolysates, exhibits key characteristics
of carcinogens (it is genotoxic).
6.4 Overall evaluation
Opium consumption is carcinogenic to
humans (Group 1).
6.5 Rationale
The evaluation of opium consumption (i.e.
smoking or ingestion) as Group 1 is based on a
determination of sufficient evidence of carcinogenicity in humans. In reaching this determination, the Working Group noted that in a cohort
study of 50  045 adults in Golestan Province, a
north-eastern province of the Islamic Republic
of Iran, self-reported opium consumption was
assessed at baseline, validated with urinary
levels of opium metabolites, and the cohort was
followed for more than a decade to ascertain incident cancers. The risk of several types of cancer –
including cancers of the urinary bladder, larynx,
and lung – was significantly higher among opium
users than non-users and increased in an exposure-dependent fashion with cumulative opium
use. The prospective cohort design minimized
concerns regarding selection bias and reverse
causation, and the detailed assessment of demographic, socioeconomic, and lifestyle factors
addressed concerns regarding the major potential
confounders of the associations of interest. These
cohort study findings are supported by multiple
case–control studies that provide evidence of
positive associations between opium consumption and these types of cancer, often based upon
larger numbers of site-specific cancer cases,
ascertained over a larger geographical area in
the Islamic Republic of Iran and, in many cases,tderived in studies that used similar exposure
assessment tools and covariate adjustments to
those used in the Golestan Cohort Study. While
individually each study has its limitations, the
Working Group concluded that, collectively,
these studies provide a basis to rule out chance,
bias, and confounding as alternative explanations for the positive association between opium
use and cancers of the urinary bladder, larynx,
and lung with reasonable certainty; thus, there
was sufficient evidence of human carcinogenicity for these three cancer types. Additionally,
evidence was deemed to be limited that opium
consumption causes cancers of the oesophagus,
stomach, pancreas, and pharynx. While positive
associations were seen in the body of evidence for
these cancers, chance, bias, and/or confounding
could not be ruled out with reasonable confidence. The sufficient evidence of carcinogenicity
in humans applies to smoking and ingestion as
routes of consumption of raw, dross, and minimally refined opium.
There is also strong evidence in experimental
systems that opium, specifically sukhteh and
opium pyrolysates (solid residues of combusted
opium), exhibits key characteristics of carcinogens. These opium forms are genotoxic.
There is inadequate evidence in experimental
animals regarding the carcinogenicity of opium
consumption because all available studies had
major qualitative or quantitative limitations


Q: What was the summary or evaluation for each cancer site?
A:"""



openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'For cancers of the urinary bladder, larynx, and lung, there was sufficient evidence of human carcinogenicity. For cancers of the oesophagus, stomach, pancreas, and pharynx, there was limited evidence that opium consumption causes these cancers.'

5. SUMMARY OF DATA REPORTED

In [10]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
5.1 Exposure characterization
Opium is a highly addictive narcotic drug
that has been used for centuries for medicinal
and non-medicinal purposes. It has analgesic,
hypnotic, antitussive, gastrointestinal, and
cognitive effects.
Opium comes from the juice (latex) of the
unripe seedpod of the poppy plant (Papaver
somniferum), and has a complex chemical
composition consisting of at least 25 alkaloids
(e.g. morphine, codeine, and thebaine) and
other ingredients. Opium is often adulterated
with compounds such as lead to enhance its
weight. Illicit opium product may therefore be
a combination of opium and other compounds.
The types and percentages of the alkaloids in
opium differ widely between different poppy
cultivars. The latex can be processed by drying
or boiling before consumption. Opium includes
raw or crude opium, opium dross (tarry residues
formed after smoking raw opium), and refined
opium or opium sap (boiled opium dross with or
without raw opium). All forms of opium are typically smoked or ingested. Pyrolysis products may
result from combustion (smoking) of all three
forms of opium. Opium derivatives (morphine,
codeine, and heroin) are not considered in the
present monograph.
Opium production and distribution have
been controlled internationally since 1961, and
190 countries have ratified an international
convention controlling the production, distribution, and use of opium.
Opium is produced illicitly in some 50 countries worldwide, and global production has
increased during the last decade from 4950 to
7610 tonnes. Over 80% of the world’s illicit opium
comes from Afghanistan. Of the total opium
produced, 15–20% is used as raw or minimally
processed opium. In 2009, the Islamic Republic
of Iran was estimated to be the world’s largest per
capita consumer of raw or minimally processed
opium, representing 42% of total global opium
consumption, followed by Afghanistan and
Pakistan. In 2018, there were an estimated 5
million users of illicit opium worldwide.
Due to its illicit nature, “street” opium is not
subject to safety standards. Legal opium is used
to produce opium tincture and syrup; however,
these represent a small proportion of global
opium production.
Epidemiological studies have been conducted
only on users of illicit forms of opium and have
used questionnaires, interviews, or patient
records to evaluate opium exposure. Some
studies compared questionnaire/interview data
on opium consumption with opium biomarkers.
Opium derivatives can be detected in blood,
5. SUMMARY OF DATA REPORTED
IARC MONOGRAPHS – 126
234
urine, hair, and nails for limited periods after
opium exposure.
The amount of detail and the quality of exposure information varied considerably across the
epidemiological studies. Some studies defined
opium consumption as opium dependence using
standardized tools such as those based on the
Diagnostic and Statistical Manual of Mental
Disorders, 4th Edition, Text Revision. Multiple
studies used the structured and validated
Golestan Cohort Study (GCS) questionnaire
(GCSQ), which defined opium exposure as use
more than once per week for at least 6 months.
The GCSQ incorporated questions on opium
type, dose, mode of consumption, temporality
of exposure, duration, intensity, and cumulative
exposure. To date, cohort studies that have used
the GCSQ have evaluated baseline questionnaire
data only, so any time-varying changes in exposure status have not been reported. This may be
important, particularly in cancers with a long
latency period. Many studies asked the participants to quantify the amount of opium consumption using grams or a local unit of nokhods per
day, but these estimates were likely imprecise.
The factors most strongly related to opium use
across studies were sex, age, tobacco smoking,
and socioeconomic status.
5.2 Cancer in humans
The studies of cancer in humans that were
available for this evaluation have all investigated illicit opium consumption in the form
of the minimally processed latex of the poppy
plant (P. somniferum). Opium, as purchased and
consumed by millions of people in several countries, is a complex mixture that includes alkaloids (e.g. morphine and thebaine), non-alkaloids
(e.g. sugars, fats, meconic acid, and water), and
impurities (e.g. lead and chromium). Several
informative cohort and case–control studies
have investigated the association between opium
consumption (by smoking or ingestion of crude
opium, opium dross, or minimally processed
opium) and cancers of various sites in humans.
Some of these studies, most notably the GCS, had
a strong design and used several strategies to alleviate concerns about bias. Cancers of the oesophagus, urinary bladder, lung, larynx, pancreas,
stomach, colon and rectum, and pharynx were
studied in more depth.
A cohort study (the GCS) showed a positive
association between opium consumption and
risk of oesophageal cancer, with an exposure–
response association. A case–control study also
showed a positive association when cases were
compared with neighbourhood controls, but
the association disappeared when cases were
compared with hospital controls. In both studies,
the large majority of the oesophageal cancer cases
were of squamous type. The Working Group
concluded that although a positive association is
credible, chance, bias, and confounding cannot
be ruled out with reasonable confidence. The
association observed in the cohort study was not
very strong and could possibly have arisen due
to residual confounding. The results of the case–
control study are subject to interpretation based
on the appropriateness of the control group.
The GCS found a strong association between
opium consumption and risk of urinary bladder
cancer, with evidence of an exposure–response
relation. Likewise, nearly all eight case–control
studies that studied the association between
opium consumption and urinary bladder
cancer found higher odds of opium use among
cases than in controls, with adjusted odds
ratios ranging from 2 to 5. Control selection,
adjustment for confounding, and definition of
exposure varied among studies; however, it was
notable that all studies, regardless of design,
pointed in the same direction. The Working
Group concluded that despite a modest number
of cases in the cohort study, a positive association
has been observed and that, collectively, the most
informative studies rule out confounding, bias,
Opium consumption
235
and reverse causation with a reasonable degree
of confidence.
The association between opium consumption and laryngeal cancer has been extensively
studied in a cohort study (the GCS) and six
case–control studies. The GCS found a strong
positive association between opium consumption
and risk of laryngeal cancer, with an exposure–
response association. Likewise, all six case–
control studies showed substantially increased
opium use among laryngeal cancer patients
compared with controls, ranging from 2- to
16-fold. The quality of these studies – including
adjustment for potential confounders, excluding
opium use that initiated within a few years before
diagnosis, and various other sensitivity analyses
– varied across studies. However, the two studies
that adjusted for many confounders and analysed
the data in various ways also found strong associations between opium consumption and laryngeal cancer. The Working Group concluded that
a positive association has been observed. The
more informative studies, collectively, rule out
chance, confounding, bias, and reverse causation with reasonable confidence. This inference
resulted from the observation of very strong
associations, exposure–response associations,
consistency across studies, availability of studies
with large sample sizes, and various efforts to
rule out bias and confounding in at least two key
studies: the GCS and IROPICAN (the Iranian
Study of Opium and Cancer).
The association between opium consumption
and lung cancer has been studied in a cohort
study (the GCS) and three case–control studies.
The cohort study found a positive association
with an exposure–response trend for increasing
quartiles of consumption. These results were
adjusted for cigarette smoking and other potential confounders, although adjustment for cigarette smoking might have been less than ideal
due to the low number of study participants with
lung cancer who used opium but never smoked
cigarettes. The quality of control selection,
adjustment, and exposure data collection varied
across the case–control studies. However, all
three case–control studies, which collectively
involved a large number of cases and controls,
showed a positive association between opium
consumption and lung cancer, with adjusted
odds ratios ranging from 2 to 6. The Working
Group concluded that a positive association has
been observed. Given the totality of evidence
and the strong association observed in the
cohort study, the Working Group concluded that
chance, bias, and confounding were unlikely to
explain the results.
In the GCS, ever-use of opium did not show
a clear association with an increased risk of
pancreatic cancer. However, there was evidence
of an association for participants who had very
high amounts of cumulative use. A case–control
study found evidence of increased risk among
opium users. The controls were from the same
clinic from which the cases were recruited,
therefore reducing the possibility of biased
reports; however, bias from data collection on
the part of interviewers cannot be entirely ruled
out. However, there was no exposure–response
association with either duration of opium use or
cumulative opium use in this case–control study.
The Working Group concluded that a credible
association was observed, but chance, bias, and
confounding cannot be ruled out, partly because
the number of studies was small. Although the
only case–control study showed some evidence to
support an association, the cohort study showed
an association only for very high exposures.
The association between opium consumption and gastric cancer has been studied in two
cohort studies (the GCS and Ardabil cohort
study) and two case–control studies. All studies
showed increased risk of gastric cancer. In one
study, opium consumers were observed to have
an increased risk of developing precursor lesions
for gastric cancer, alleviating concerns about
reverse causation. The GCS results showed a
positive association with opium consumption,
IARC MONOGRAPHS – 126
236
particularly for the noncardia subtype, but the
exposure–response trend was not statistically
significant. In the Ardabil cohort study, opium
consumption was associated with an increased
risk of baseline antral and gastric body intestinal
metaplasia, which are precursor lesions for gastric
cancer, and subsequent incident gastric cancer.
Both case–control studies also showed strong
positive associations between opium consumption and odds of gastric cancer. The Working
Group’s assessment was that although a positive
association in the body of evidence was credible,
chance, confounding, and bias from potential
under-reporting in case–control studies cannot
be ruled out with reasonable confidence. This
decision was reached partly because of lack of
a clear exposure−response relation and partly
because of lack of data for H. pylori or diet in some
of these studies, which may lead to confounded
results.
The association between opium consumption
and cancers of the colon and rectum has been
studied in a cohort study (the GCS) and two
case–control studies. The GCS found no positive association between opium consumption
and risk of colon cancer, nor did it find an association with cumulative opium consumption.
However, two case–control studies with similar
design, conducted by some of the same investigators, found evidence of strong associations
between opium consumption and risk of cancer
of the colon and rectum. The Working Group
concluded that an association has not been established, primarily because of conflicting evidence
and the lack of any positive association in the
cohort study.
Only one case–control study was considered
informative for cancers of the head and neck
excluding the larynx. This study had a large
sample size and used a variety of methods to
adjust for confounding and alleviate concerns
about reverse causation and under-reporting
by cases and controls. This study found a
strong positive association between opium
consumption and cancer of the pharynx. The
Working Group concluded that a positive association between opium consumption and pharyngeal cancer was credible, but that chance, bias,
and confounding could not be excluded with
reasonable confidence, primarily because there
was only one (well-conducted) study.
5.3 Cancer in experimental animals
The available studies on opium could not be
interpreted as showing either the presence or the
absence of a carcinogenic effect because of major
limitations, including low numbers of animals,
lack of survival and body-weight data, unknown
adequacy of the treatment doses, and limited
reporting.
5.4 Mechanistic evidence
Evidence of opium absorption, distribution, and metabolism in humans and rodents is
provided by studies on intoxication and excretion. In humans, opium metabolites have been
detected in urine, hair, and blood after ingestion
or smoking of opium. In rats, metabolites have
been measured in hair after oral exposure and in
urine after inhalation of volatilized opium.
There is consistent and coherent evidence
in experimental systems that opium exhibits
key characteristics of carcinogens: it is genotoxic. There is consistent evidence for sukhteh
(opium dross) and opium pyrolysates (solid
residues of combusted opium), but not for raw
opium, that such forms induce clastogenicity and
mutagenicity. Studies in exposed humans were
uninformative. Sukhteh and opium pyrolysates
induced dose-related increases in sister-chromatid exchange in human primary peripheral
blood mononuclear cells and Chinese hamster
ovary cells, with and without metabolic activation. No data for mammalian experimental
systems in vivo were available. Sukhteh and opium
Opium consumption
237
pyrolysates consistently induced mutagenicity
in multiple strains of Salmonella typhimurium
that are indicators of base-pair substitution and
frameshift mutations. The studies on genetic
and related effects are not numerous. However,
consistent findings were seen across several test
systems in different species, indicating that forms
of opium containing pyrolysates are genotoxic.
In multiple experimental systems, including
human cells and cells from another mammalian
and non-mammalian species, forms of opium
containing pyrolysates induced both clastogenicity and mutagenicity. This evidence is also
coherent with what is known about the genotoxicity and mutagenicity of combustion products.
There is also some evidence of increases in
the incidence of precursors to chronic inflammation; however, the data are inconsistent. Evidence
pertaining to oxidative stress and antioxidant
status is conflicting. Information related to other
key characteristics of carcinogens was sparse.


Q: Which key characteristics of carcinogens presented the consumption of opium?
A:"""


openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Opium exhibits key characteristics of carcinogens, including genotoxicity, clastogenicity, and mutagenicity.'